### Defining costs

In [5]:
kulud = {
  "Kompleksteenus(€/tm)": 15,
  "Transport (€/tm)": 6,
  "Alghinna(%)": 10
} 

### Reading in EXCEL file for wood prices.

In [6]:
import pandas as pd

# Reading in EXCEL file for wood prices.
wood_prices = pd.read_excel('Data_Sources/Hinnakiri.xlsx')

print("Hinnakiri:")
print(wood_prices)

Hinnakiri:
                    Sortiment  Hind (€/tm)
0                     Ma palk           95
1                     Ku palk           95
2                Ks palk/pakk          110
3                     Hb palk           58
4                     Lm palk           60
5                     Lv palk           50
6                     Sa palk           40
7                     Ta palk           40
8                     Lh palk           40
9                     Sd palk           40
10  Lehtpuu palk (Va, Ja, Kp)           35
11                Ma peenpalk           64
12                Ku peenpalk           64
13              Ma paberipuit           62
14              Ku paberipuit           62
15              Ks paberipuit           50
16              Hb paberipuit           50
17                  Küttepuit           39
18                    Jäätmed            3


In [7]:
import requests
import json
from urllib.parse import quote

# Authorization API endpoint
auth_url = "https://lindaforest.collectivecrunch.net/api/v1/auth"

# Authorization headers with API keys
headers = {
    'linda-universal-api-key-id': '',
    'linda-universal-api-key-secret': ''
}

try:
    # Send POST request with authorization headers
    auth_response = requests.post(auth_url, headers=headers, timeout=30)
    auth_response.raise_for_status()  # Gives out error if authorization fails
    auth_token = auth_response.json()['accessToken']
    print("Authorization successful.")

    # ---- SIMPLE VARIABLES (change here) ----
    country = "ee"                   # 'ee' or 'lv'
    property_id = "33801:001:1133"   # katastritunnus (':' kodeeritakse automaatselt)

    include_stands = True            # include stands
    include_predictions = True      # include predictions
    include_geometries = False       # include geometries (EPSG:4326)
    # ---------------------------------------

    # Build endpoint and query parameters
    base = "https://lindaforest.collectivecrunch.net/api/v1"
    property_encoded = quote(property_id, safe="")
    endpoint = f"{base}/scout/{country}/property/{property_encoded}"

    # API is waiting for 'true'/'false' string
    params = {
        "include_stands": str(include_stands).lower(),
        "include_predictions": str(include_predictions).lower(),
        "include_geometries": str(include_geometries).lower(),
    }

    # To get the data use Bearer token
    data_headers = {
        'Authorization': f'Bearer {auth_token}',
        'Accept': 'application/json'
    }

    # Get the data
    data_response = requests.get(endpoint, headers=data_headers, params=params, timeout=60)
    print("Request URL:", data_response.url)  # See the exact URL with parameters
    data_response.raise_for_status()  # Gives out error if data request fails
    data = data_response.json()
    print("Data successfully retrieved:", json.dumps(data, indent=2, ensure_ascii=False))

except requests.exceptions.HTTPError as err:
    print(f"HTTP Error: {err}")
    if err.response is not None:
        print(f"Response content: {err.response.text}")
except requests.exceptions.RequestException as err:
    print(f"Unexpected error: {err}")

HTTP Error: 401 Client Error: Unauthorized for url: https://lindaforest.collectivecrunch.net/api/v1/auth
Response content: {"errorCode":"unathorized","status":401,"name":"UnauthorizedError","message":"error.apiKey.invalid"}


### Reading in the API file for a specific cadastral.

In [8]:
import json

# Reading in JSON file for a specific cadastral.
e = data['stands']

# Create an empty list to store the new, transformed data records.
transformed_data_list = []
# Loop through each individual record (which is a dictionary) in the list.
for record in e:
    # Extract the stand number for the current record.
    stand_number = record["stand_number"]
    # Now you can safely access the values using keys from this single record.
    # Create a list of dictionaries for each species in the current record.
    species_data = [
        {
            "Eraldise nr": stand_number,
            "Puuliik": "Pine",
            "Kõrgus m": record["pine_bam_height_m"],
            "Diameeter cm": record["pine_bam_dbh_cm"],
            "Pindala ha": record["total_area_ha"],
            "Tihedus m3/ha": record["pine_total_volume_m3_ha"],
            "Tagavara m3": record["total_area_ha"] * (record["pine_total_volume_m3_ha"] or 0)
        },
        {
            "Eraldise nr": stand_number,
            "Puuliik": "Spruce",
            "Kõrgus m": record["spruce_bam_height_m"],
            "Diameeter cm": record["spruce_bam_dbh_cm"],
            "Pindala ha": record["total_area_ha"],
            "Tihedus m3/ha": record["spruce_total_volume_m3_ha"],
            "Tagavara m3": record["total_area_ha"] * (record["spruce_total_volume_m3_ha"] or 0)
        },
        {
            "Eraldise nr": stand_number,
            "Puuliik": "Birch",
            "Kõrgus m": record["birch_bam_height_m"],
            "Diameeter cm": record["birch_bam_dbh_cm"],
            "Pindala ha": record["total_area_ha"],
            "Tihedus m3/ha": record["birch_total_volume_m3_ha"],
            "Tagavara m3": record["total_area_ha"] * (record["birch_total_volume_m3_ha"] or 0)
        
        },
        {
            "Eraldise nr": stand_number,
            "Puuliik": "Other Deciduous",
            "Kõrgus m": record["other_deciduous_bam_height_m"],
            "Diameeter cm": record["other_deciduous_bam_dbh_cm"],
            "Pindala ha": record["total_area_ha"],
            "Tihedus m3/ha": record["other_deciduous_total_volume_m3_ha"],
            "Tagavara m3": record["total_area_ha"] * (record["other_deciduous_total_volume_m3_ha"] or 0),
            
        }
    ]
    # Add the newly created species data to our main list.
    transformed_data_list.extend(species_data)
# Finally, create a single DataFrame from the combined list of all records.
API_cadastral_data = pd.DataFrame(transformed_data_list)
print(API_cadastral_data)

NameError: name 'data' is not defined

### Reading in EXCEL file for tree names.

In [ ]:
# Reading in EXCEL file for tree names.
tree_name = pd.read_excel('Data_Sources/Puu_nimetused_EE_ENG.xlsx')

print("Puude nimetused EE/ENG:")
print(tree_name.head())


Puude nimetused EE/ENG:
  Name_EE         Name_ENG
0      MA             Pine
1      KU           Spruce
2      KS            Birch
3      LV  Other Deciduous
4      HB              NaN


### Reading in EXCEL file for relative heights.

In [ ]:
# Reading in EXCEL file for relative height.
relative_heights = pd.read_excel('Data_Sources/Suhtelised_tugikõrgused.xlsx')

print("Suhtelised tugikõrgused:")
print(relative_heights)


Suhtelised tugikõrgused:
     d      MA     KU      KS      LV      HB      LM      TA      SA      VA  \
0    8  0.4300  0.350  0.5500  0.5500  0.5200  0.5200  0.3800  0.3800  0.3800   
1    9  0.5000  0.410  0.6000  0.6000  0.5700  0.5700  0.4400  0.4400  0.4400   
2   10  0.5500  0.470  0.6400  0.6400  0.6200  0.6200  0.5000  0.5000  0.5000   
3   11  0.6100  0.530  0.6800  0.6800  0.6600  0.6600  0.5500  0.5500  0.5500   
4   12  0.6500  0.580  0.7200  0.7200  0.7000  0.7000  0.6000  0.6000  0.6000   
5   13  0.7000  0.630  0.7600  0.7600  0.7400  0.7400  0.6500  0.6500  0.6500   
6   14  0.7400  0.680  0.7900  0.7900  0.7700  0.7700  0.7000  0.7000  0.7000   
7   15  0.7700  0.720  0.8200  0.8200  0.8000  0.8000  0.7400  0.7400  0.7400   
8   16  0.8100  0.760  0.8400  0.8400  0.8300  0.8300  0.7700  0.7700  0.7700   
9   17  0.8400  0.800  0.8700  0.8700  0.8600  0.8600  0.8100  0.8100  0.8100   
10  18  0.8700  0.830  0.8900  0.8900  0.8800  0.8800  0.8400  0.8400  0.8400   
11 

### Reading in EXCEL file for log volume distribution

In [ ]:
# Reading in EXCEL file for type proportions.
log_volume_distribution = pd.read_excel('Data_Sources/Mahutabel.xlsx')

print("Mahutabel:")
print(log_volume_distribution)

Mahutabel:
     d klass+pl+h24 x m  kõrgus      palk     peenp     paber      küte  \
0                 8MA16     8.0  0.000000  0.000000  0.000000  0.500000   
1                12MA16    11.0  0.000000  0.000000  0.714286  0.000000   
2                16MA16    13.0  0.000000  0.333333  0.400000  0.000000   
3                20MA16    15.0  0.000000  0.680000  0.080000  0.000000   
4                24MA16    16.0  0.289474  0.368421  0.105263  0.000000   
...                 ...     ...       ...       ...       ...       ...   
3205             36PN27    32.0  0.000000  0.000000  0.000000  0.794521   
3206             40PN27    33.0  0.000000  0.000000  0.000000  0.795699   
3207             44PN27    34.0  0.000000  0.000000  0.000000  0.800000   
3208             48PN27    35.0  0.000000  0.000000  0.000000  0.806452   
3209             52PN27    35.0  0.000000  0.000000  0.000000  0.804805   

       jäätmed  kokku  
0     0.500000    1.0  
1     0.285714    1.0  
2     0.266667  

### Merging dataframes API_cadastral_data and tree_name.

In [ ]:
# Merging two dataframes on main species names and adding Name_EE, Name_ENG.
cadastral_data = pd.merge(API_cadastral_data, tree_name, left_on='Puuliik', right_on='Name_ENG', how='left')

### Adding a 'Relative height' and 'h24' column to the table.

In [ ]:
import numpy as np

cadastral_data['Suhteline tugikõrgus'] = np.nan

for index, row in cadastral_data.iterrows():
    name_ee = row['Name_EE']
    diameter_cm = row['Diameeter cm']
    try:
        # Look up the correct value from relative_heights table.
        h24_value = relative_heights.loc[relative_heights['d'] == diameter_cm, name_ee]
        # When the value is found add it to the column 
        if not h24_value.empty:
            cadastral_data.at[index, 'Suhteline tugikõrgus'] = h24_value.values[0]
    except KeyError:
        # When there isn´t a value add NaN to the row
        pass

# Adding a 'h24' column to the table.
cadastral_data['h24'] = cadastral_data.apply(
    lambda row: 16 if (pd.notna(row['Kõrgus m']) and pd.notna(row['Suhteline tugikõrgus']) and row['Suhteline tugikõrgus'] != 0 and (np.ceil(row['Kõrgus m'] / row['Suhteline tugikõrgus'])) < 16) else (int(np.ceil(row['Kõrgus m'] / row['Suhteline tugikõrgus'])) if pd.notna(row['Kõrgus m']) and pd.notna(row['Suhteline tugikõrgus']) and row['Suhteline tugikõrgus'] != 0 else np.nan),
    axis=1)
#
cadastral_data['h24'] = cadastral_data['h24'].astype('Int64')

### Adding a 'Diameetri klass' column to the table

In [ ]:
# Adding a 'Diameetri klass' column to the table.
def diameter_category(diameter_cm):
    if 5 <= diameter_cm <= 52:
        # Round up to the nearest multiple of 4 (aligned with 8, 12, 16, …, 52)
        return ((diameter_cm + 3) // 4) * 4
    elif diameter_cm > 52:
        return 52
    return None
cadastral_data['Diameetri klass'] = (
    cadastral_data['Diameeter cm']
    .apply(diameter_category)# Using .apply() to use the function on 'Diameeter cm' values.
    .astype('Int64')# Getting 'Diameetri klass' column to Int64 type.
)

### Adding a 'Sortimendi jaotusklass' column to the table

In [ ]:
cadastral_data['Sortimendi jaotusklass'] = cadastral_data['Diameetri klass'].astype(str) + '' + cadastral_data['Name_EE'].astype(str) + '' + cadastral_data['h24'].astype(str)

### Merging log_volume_distribution with cadastral_data on 'Sortimendi jaotusklass' to get the Mahu jaotus.

In [ ]:
cadastral_data = pd.merge(cadastral_data, 
                     log_volume_distribution, 
                     left_on='Sortimendi jaotusklass', 
                     right_on='d klass+pl+h24 x m', 
                     how='inner')

# Calculating proportions for each row
columns_to_multiply = ['palk', 'peenp', 'paber', 'küte', 'jäätmed']

# A loop that multiplys type proportion columns by 'Tagavara m3' column values.
for column in columns_to_multiply:
    cadastral_data[column] = cadastral_data[column] * cadastral_data['Tagavara m3']

# Dropping unnecessary columns from the table.
cadastral_data = cadastral_data.drop(columns=['d klass+pl+h24 x m', 'kõrgus', 'kokku','Name_ENG'])

print("Lõplik katastriandmete tabel:")
print(cadastral_data)

Lõplik katastriandmete tabel:
   Eraldise nr          Puuliik  Kõrgus m  Diameeter cm  Pindala ha  \
0            0             Pine      14.8            23        1.04   
1            0           Spruce      18.0            23        1.04   
2            0            Birch      15.1            17        1.04   
3            0  Other Deciduous       8.6            11        1.04   
4            1             Pine      22.8            30        1.42   
5            1           Spruce      23.0            28        1.42   
6            1            Birch      22.0            22        1.42   
7            2             Pine      23.2            33        1.94   
8            2           Spruce      23.0            29        1.94   
9            2            Birch      22.7            22        1.94   
10           2  Other Deciduous      23.3            28        1.94   
11           3             Pine      24.4            32        1.65   
12           3           Spruce      24.9      

### Creating a new dataframe to sum up 'Maht' and 'Hind'

In [ ]:
# Define the mapping between Sortiment, source (Name_EE + column), and price names
mappings = [
    ("Ma palk",        ("MA", "palk"),   "Ma palk"),
    ("Ku palk",        ("KU", "palk"),   "Ku palk"),
    ("Ks palk/pakk",   ("KS", "palk"),   "Ks palk/pakk"),
    ("Teised liigid/Lv palk", ("LV", "palk"), "Lv palk"),
    ("Ma peenpalk",    ("MA", "peenp"),  "Ma peenpalk"),
    ("Ku peenpalk",    ("KU", "peenp"),  "Ku peenpalk"),
    ("Ma paberipuit",  ("MA", "paber"),  "Ma paberipuit"),
    ("Ku paberipuit",  ("KU", "paber"),  "Ku paberipuit"),
    ("Ks paberipuit",  ("KS", "paber"),  "Ks paberipuit"),
    ("Küttepuit",      (None, "küte"),   "Küttepuit"),
    ("Jäätmed",        (None, "jäätmed"),"Jäätmed"),
]
rows = []
for sortiment, (name_ee, col), price_name in mappings:
    if name_ee:  # sum by Name_EE + column
        volume = cadastral_data.loc[cadastral_data["Name_EE"] == name_ee, col].sum()
    else:        # global sum
        volume = cadastral_data[col].sum()
    # find matching price
    price = wood_prices.loc[wood_prices["Sortiment"] == price_name, "Hind (€/tm)"].squeeze()
    rows.append({
        "Sortiment": sortiment,
        "Maht (tm)": volume,
        "Summa (€)": volume * price
    })
# Create DataFrame
total_volume_and_amount = pd.DataFrame(rows)
# Add total row
total_volume_and_amount.loc[len(total_volume_and_amount)] = {
    "Sortiment": "Kokku",
    "Maht (tm)": total_volume_and_amount["Maht (tm)"].sum(),
    "Summa (€)": total_volume_and_amount["Summa (€)"].sum()
}
# Round results
total_volume_and_amount["Maht (tm)"] = total_volume_and_amount["Maht (tm)"].round(1)
total_volume_and_amount["Summa (€)"] = total_volume_and_amount["Summa (€)"].round(1)

print("Lõplik mahu ja hinna tabel:")
print(total_volume_and_amount)

Lõplik mahu ja hinna tabel:
                Sortiment  Maht (tm)  Hind (€)
0                 Ma palk      113.4   10772.9
1                 Ku palk      622.1   59103.4
2            Ks palk/pakk      193.6   21296.7
3   Teised liigid/Lv palk       25.1    1254.6
4             Ma peenpalk       15.4     985.8
5             Ku peenpalk      119.0    7614.8
6           Ma paberipuit        8.8     544.4
7           Ku paberipuit       38.1    2363.0
8           Ks paberipuit      201.1   10057.5
9               Küttepuit       31.0    1207.6
10                Jäätmed      342.1    1026.4
11                  Kokku     1709.8  116227.2


### Creating a new dataframe to get recommended starting bid

In [ ]:
# Extract key values
maht_kokku = total_volume_and_amount.at[total_volume_and_amount.index[total_volume_and_amount['Sortiment'] == 'Kokku'][0], 'Maht (tm)']
hind_kokku = total_volume_and_amount.at[total_volume_and_amount.index[total_volume_and_amount['Sortiment'] == 'Kokku'][0], 'Summa (€)']
maht_jaatmeteta = total_volume_and_amount.loc[
    ~total_volume_and_amount['Sortiment'].isin(['Jäätmed', 'Kokku']),
    'Maht (tm)'
].sum()
# Calculations
kulud_tm = kulud["Kompleksteenus(€/tm)"] + kulud["Transport (€/tm)"]
kulud_jaatmeteta = maht_jaatmeteta * kulud_tm
tulud_kulud_jaatmeteta = hind_kokku - kulud_jaatmeteta
soovituslik_alghind = tulud_kulud_jaatmeteta * (1 - kulud["Alghinna(%)"] / 100)
# Build result table
results = {
    "Maht kokku":              (maht_kokku, "tm"),
    "Tulud kokku":              (hind_kokku, "€"),
    "Kulud (jäätmeteta)":      (kulud_jaatmeteta, "€"),
    "Tulud-Kulud (jäätmeteta)":(tulud_kulud_jaatmeteta, "€"),
    "Soovituslik alghind":     (soovituslik_alghind, "€"),
}
cadastral_data_summary = (
    pd.DataFrame.from_dict(results, orient="index", columns=["Väärtus", "Ühik"])
    .assign(Väärtus=lambda df: df["Väärtus"].round(1))
)

print("Katastriüksuse hinnanguline väärtus:")
print(cadastral_data_summary)

Katastriüksuse hinnanguline väärtus:
                           Väärtus Ühik
Maht kokku                  1709.8   tm
Tulud kokku               116227.2    €
Kulud (jäätmeteta)         28719.6    €
Tulud-Kulud (jäätmeteta)   87507.6    €
Soovituslik alghind        78756.8    €


### Cadastral summary in Excel.

In [ ]:
#with pd.ExcelWriter('Cadastral_summary.xlsx') as writer:
    #cadastral_data.to_excel(writer, sheet_name='Katastriandmed', index=False)
    #total_volume_and_amount.to_excel(writer, sheet_name='Maht ja hind', index=False)
    #cadastral_data_summary.to_excel(writer, sheet_name='Kokkuvõte')